<a href="https://colab.research.google.com/github/Advanced-Research-Centre/HilbertCorps/blob/main/Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
%%capture
!pip3 install qiskit
!pip3 install qiskit-aer
!pip3 install --upgrade matplotlib
# !pip3 install gymnasium
# !pip3 install pylatexenc

In [103]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Statevector
import numpy as np
import copy
import matplotlib.pyplot as plt
from numpy.ma.extras import average
from statistics import mean
simulator = Aer.get_backend('statevector_simulator')
%matplotlib inline

In [3]:
k = 1         # Number of qubits of information
n = 3         # Dimension of encoding space
s = n - k     # Maximum number of syndrome measurements allowed

perr_x = 0
perr_y = 0
perr_z = 0

trials = 1    # How many random input quantum states will be tested for the QECC scheme
shots = 1000  # For penalty precision

def prep_state(qc,ax,ay):
    qc.rx(ax, range(k))
    qc.ry(ay, range(k))
    return qc

def add_enc_circ(ax,ay):
    qc = QuantumCircuit(n+s, k)
    qc.rx(ax, range(k))
    qc.ry(ay, range(k))
    qc.barrier()
    for i in range(0,n-k):  # Currently works only for k = 1. For higher k, block encoding?
        qc.cx(k-1,k+i)
    qc.barrier()
    return qc

def add_err_circ(qc):
    # tgt = np.random.randint(0,n)
    # px = np.random.rand()
    # if px < perr_x:
    #     qc.x(tgt)
    for i in range(n):
        px = np.random.rand()
        py = np.random.rand()
        pz = np.random.rand()
        error_indices = []
        if px < perr_x:
            qc.x(i)
        if py < perr_y:
            qc.y(i)
        if pz < perr_z:
            qc.z(i)
    qc.barrier()
    return qc

def add_dec_circ(qc):
    for i in range(0,n-k):  # Currently works only for k = 1. For higher k, block encoding?
        qc.cx(k-1,k+i)
    qc.barrier()
    return qc

In [22]:
def run_episode(syn_qc):
    penalties = []
    # np.random.seed(1)           # REMOVE later, now for testing

    for trial in range(trials):

        qc = QuantumCircuit(k, k)
        ax = np.random.rand()*2*np.pi
        ay = np.random.rand()*2*np.pi

        q_state = prep_state(qc, ax, ay)
        # print(Statevector(q_state))
        q_state.measure(range(k), range(k))

        result = execute(q_state, simulator, shots=shots).result()
        m1 = result.get_counts(q_state)
        # print(m1)

        ec = add_enc_circ(ax, ay)

        err_trials = 1       # How many times a specific choice of syndrome circuit is tested for different random errors, bigger the better (and slower)

        for _ in range(err_trials):

            enc_circ = copy.deepcopy(ec)

            err_circ = add_err_circ(enc_circ)

            syn_circ = err_circ.compose(syn_qc)

            dec_circ = add_dec_circ(syn_circ)
            # print(dec_circ)
            # print(Statevector(dec_circ))
            dec_circ.measure(range(k), range(k))

            result = execute(dec_circ, simulator, shots=shots).result()
            m2 = result.get_counts(dec_circ)
            # print(m2)


            penalty = sum(abs(m1.get(key, 0) - m2.get(key, 0))/shots for key in set(m1) | set(m2))  # This is the agent's penalty
            # print(set(m1)|set(m2),m1.get('0'),m2.get('0'),penalty)

            penalties.append(penalty)

    return penalties  #, best_penalty, best_syndrome_circuit

In [106]:
def add_syn_circ():
    qc = QuantumCircuit(n+s, k)
    qc.cx(0,3)
    qc.cx(1,3)
    qc.cx(1,4)
    qc.cx(2,4)
    qc.barrier()
    qc.x(4)
    qc.mcx([3,4],0)
    qc.x(4)
    qc.mcx([3,4],1)
    qc.x(3)
    qc.mcx([3,4],2)
    qc.x(3)
    qc.barrier()
    return qc

def gen_syn_circ(a_hist, p_hist):
    rnd_syn = np.random.rand()
    if rnd_syn < 0.4:
      syn_circ = add_syn_circ()
    else:
      syn_circ = QuantumCircuit(n+s, k)
    return syn_circ
    # qc = QuantumCircuit(n+s, k)
    # num_syndrome_gates = 4
    # num_recovery_gates = 3
    # # Agent's action Part 1
    # for _ in range(num_syndrome_gates):
    #     data_qubit = np.random.randint(n)
    #     syndrome_qubit = np.random.randint(n, n + s)
    #     qc.cx(data_qubit, syndrome_qubit)
    # # Agent's action Part 2
    # for _ in range(num_recovery_gates):
    #     target_qubit = np.random.randint(n)
    #     num_controls = np.random.randint(1, s + 1)
    #     control_qubits = list(np.random.choice(np.arange(n, n + s), size=num_controls, replace=False))    # Change to inverted control
    #     qc.cx(control_qubits, target_qubit)
    # return qc

In [ ]:
# Test variance of penalty with perr_x

trials = 20     # How many random input quantum states will be tested for the QECC scheme
shots = 1000    # For penalty precision
syn_trails = 1  # Number of changes agent gets to evolve/generate a syndrome circuit
best_avg_penalty = float('inf')
best_syndrome_circuit = None
avg_penalty_list = []
syn_trial_list = []
perr_y = 0
perr_z = 0
perr_x_test = np.linspace(0,1,50)
for perr_x_i in perr_x_test:
  perr_x = perr_x_i
  for t in range(syn_trails):
      syn_circ = gen_syn_circ(syn_trial_list,avg_penalty_list)      # Action of the agent (Bit flip code / Random / Evolutionary)
      syn_trial_list.append(syn_circ)
      penalties = run_episode(syn_circ)                             # Interaction with the environment, performs the action on the environment
      avg_penalty = np.mean(penalties)                              # Penalty
      avg_penalty_list.append(avg_penalty)
      if avg_penalty < best_avg_penalty:
          best_avg_penalty = avg_penalty
          best_syndrome_circuit = copy.deepcopy(syn_circ)

print(avg_penalty_list)

In [ ]:
# Plot variance of penalty with perr_x

plt.plot(avg_penalty_list)
plt.ylim(0,1)
plt.show()

In [110]:
# Syndrome construction

perr_x = 0.1
perr_y = 0
perr_z = 0

trials = 20     # How many random input quantum states will be tested for the QECC scheme
shots = 1000    # For penalty precision

syn_trails = 5  # Number of changes agent gets to evolve/generate a syndrome circuit
best_avg_penalty = float('inf')
best_syndrome_circuit = None
avg_penalty_list = []
syn_trial_list = []

for t in range(syn_trails):
    syn_circ = gen_syn_circ(syn_trial_list,avg_penalty_list)      # Action of the agent (Bit flip code / Random / Evolutionary)
    syn_trial_list.append(syn_circ)
    penalties = run_episode(syn_circ)                             # Interaction with the environment, performs the action on the environment
    avg_penalty = np.mean(penalties)                              # Penalty
    avg_penalty_list.append(avg_penalty)
    print(avg_penalty,syn_circ)
    if avg_penalty < best_avg_penalty:
        best_avg_penalty = avg_penalty
        best_syndrome_circuit = copy.deepcopy(syn_circ)

0.11779999999999997      
q_0: 
     
q_1: 
     
q_2: 
     
q_3: 
     
q_4: 
     
c: 1/
     
0.027200000000000002                           ░      ┌───┐                          ░ 
q_0: ──■──────────────────░──────┤ X ├──────────────────────────░─
       │                  ░      └─┬─┘     ┌───┐                ░ 
q_1: ──┼────■────■────────░────────┼───────┤ X ├────────────────░─
       │    │    │        ░        │       └─┬─┘     ┌───┐      ░ 
q_2: ──┼────┼────┼────■───░────────┼─────────┼───────┤ X ├──────░─
     ┌─┴─┐┌─┴─┐  │    │   ░        │         │  ┌───┐└─┬─┘┌───┐ ░ 
q_3: ┤ X ├┤ X ├──┼────┼───░────────■─────────■──┤ X ├──■──┤ X ├─░─
     └───┘└───┘┌─┴─┐┌─┴─┐ ░ ┌───┐  │  ┌───┐  │  └───┘  │  └───┘ ░ 
q_4: ──────────┤ X ├┤ X ├─░─┤ X ├──■──┤ X ├──■─────────■────────░─
               └───┘└───┘ ░ └───┘     └───┘                     ░ 
c: 1/═════════════════════════════════════════════════════════════
                                                                  
0.08770000

In [111]:
print(best_syndrome_circuit)
print(avg_penalty_list)

                          ░      ┌───┐                          ░ 
q_0: ──■──────────────────░──────┤ X ├──────────────────────────░─
       │                  ░      └─┬─┘     ┌───┐                ░ 
q_1: ──┼────■────■────────░────────┼───────┤ X ├────────────────░─
       │    │    │        ░        │       └─┬─┘     ┌───┐      ░ 
q_2: ──┼────┼────┼────■───░────────┼─────────┼───────┤ X ├──────░─
     ┌─┴─┐┌─┴─┐  │    │   ░        │         │  ┌───┐└─┬─┘┌───┐ ░ 
q_3: ┤ X ├┤ X ├──┼────┼───░────────■─────────■──┤ X ├──■──┤ X ├─░─
     └───┘└───┘┌─┴─┐┌─┴─┐ ░ ┌───┐  │  ┌───┐  │  └───┘  │  └───┘ ░ 
q_4: ──────────┤ X ├┤ X ├─░─┤ X ├──■──┤ X ├──■─────────■────────░─
               └───┘└───┘ ░ └───┘     └───┘                     ░ 
c: 1/═════════════════════════════════════════════════════════════
                                                                  
[0.11779999999999997, 0.027200000000000002, 0.08770000000000001, 0.20050000000000004, 0.0708]
